In [182]:
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.model_selection import cross_val_score


## Read Data

In [183]:
df_train = pd.read_csv("../datasets/train.csv")
df_test = pd.read_csv("../datasets/test.csv")

# รวมข้อมูล
df = pd.concat([df_train, df_test], ignore_index=True)
df_copy = df.copy()
df_copy.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


## Understanding Data

In [184]:
df_copy.keys()

Index(['Unnamed: 0', 'id', 'Gender', 'Customer Type', 'Age', 'Type of Travel',
       'Class', 'Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes',
       'satisfaction'],
      dtype='object')

In [185]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129880 entries, 0 to 129879
Data columns (total 25 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Unnamed: 0                         129880 non-null  int64  
 1   id                                 129880 non-null  int64  
 2   Gender                             129880 non-null  object 
 3   Customer Type                      129880 non-null  object 
 4   Age                                129880 non-null  int64  
 5   Type of Travel                     129880 non-null  object 
 6   Class                              129880 non-null  object 
 7   Flight Distance                    129880 non-null  int64  
 8   Inflight wifi service              129880 non-null  int64  
 9   Departure/Arrival time convenient  129880 non-null  int64  
 10  Ease of Online booking             129880 non-null  int64  
 11  Gate location                      1298

In [186]:
df_copy.describe().round(2)

,Unnamed: 0,id,Age,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
count,129880.00,129880.00,129880.00,129880.00,129880.00,129880.00,129880.00,129880.00,129880.00,129880.00,129880.00,129880.00,129880.00,129880.00,129880.00,129880.00,129880.00,129880.00,129880.00,129487.00
mean,44158.70,64940.50,39.43,1190.32,2.73,3.06,2.76,2.98,3.20,3.25,3.44,3.36,3.38,3.35,3.63,3.31,3.64,3.29,14.71,15.09
std,31207.38,37493.27,15.12,997.45,1.33,1.53,1.40,1.28,1.33,1.35,1.32,1.33,1.29,1.32,1.18,1.27,1.18,1.31,38.07,38.47
min,0.00,1.00,7.00,31.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00
25%,16234.75,32470.75,27.00,414.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,3.00,3.00,3.00,2.00,0.00,0.00
50%,38963.50,64940.50,40.00,844.00,3.00,3.00,3.00,3.00,3.00,3.00,4.00,4.00,4.00,4.00,4.00,3.00,4.00,3.00,0.00,0.00
75%,71433.25,97410.25,51.00,1744.00,4.00,4.00,4.00,4.00,4.00,4.00,5.00,4.00,4.00,4.00,5.00,4.00,5.00,4.00,12.00,13.00
max,103903.00,129880.00,85.00,4983.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1592.00,1584.00


In [187]:
df_copy.isnull().sum()

Unnamed: 0                             0
id                                     0
Gender                                 0
Customer Type                          0
Age                                    0
Type of Travel                         0
Class                                  0
Flight Distance                        0
Inflight wifi service                  0
Departure/Arrival time convenient      0
Ease of Online booking                 0
Gate location                          0
Food and drink                         0
Online boarding                        0
Seat comfort                           0
Inflight entertainment                 0
On-board service                       0
Leg room service                       0
Baggage handling                       0
Checkin service                        0
Inflight service                       0
Cleanliness                            0
Departure Delay in Minutes             0
Arrival Delay in Minutes             393
satisfaction    

In [188]:
df_copy_cleaned = df_copy.dropna()
df_copy_cleaned=df_copy_cleaned.drop(['Unnamed: 0','id'],axis=1).reset_index(drop=True)
df_copy_cleaned


,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129482,Male,disloyal Customer,34,Business travel,Business,526,3,3,3,1,...,4,3,2,4,4,5,4,0,0.0,neutral or dissatisfied
129483,Male,Loyal Customer,23,Business travel,Business,646,4,4,4,4,...,4,4,5,5,5,5,4,0,0.0,satisfied
129484,Female,Loyal Customer,17,Personal Travel,Eco,828,2,5,1,5,...,2,4,3,4,5,4,2,0,0.0,neutral or dissatisfied
129485,Male,Loyal Customer,14,Business travel,Business,1127,3,3,3,3,...,4,3,2,5,4,5,4,0,0.0,satisfied


In [189]:
df_copy_cleaned.isnull().sum()

Gender                               0
Customer Type                        0
Age                                  0
Type of Travel                       0
Class                                0
Flight Distance                      0
Inflight wifi service                0
Departure/Arrival time convenient    0
Ease of Online booking               0
Gate location                        0
Food and drink                       0
Online boarding                      0
Seat comfort                         0
Inflight entertainment               0
On-board service                     0
Leg room service                     0
Baggage handling                     0
Checkin service                      0
Inflight service                     0
Cleanliness                          0
Departure Delay in Minutes           0
Arrival Delay in Minutes             0
satisfaction                         0
dtype: int64

In [190]:
df_copy_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129487 entries, 0 to 129486
Data columns (total 23 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Gender                             129487 non-null  object 
 1   Customer Type                      129487 non-null  object 
 2   Age                                129487 non-null  int64  
 3   Type of Travel                     129487 non-null  object 
 4   Class                              129487 non-null  object 
 5   Flight Distance                    129487 non-null  int64  
 6   Inflight wifi service              129487 non-null  int64  
 7   Departure/Arrival time convenient  129487 non-null  int64  
 8   Ease of Online booking             129487 non-null  int64  
 9   Gate location                      129487 non-null  int64  
 10  Food and drink                     129487 non-null  int64  
 11  Online boarding                    1294

## OneHotEncoder

In [191]:
encoded_need=[]
for i in df_copy_cleaned.columns:
  if df_copy_cleaned[i].dtype=='object':
    print(f"Unique values of '{i}' is {df_copy_cleaned[i].unique()}")
    encoded_need.append(i)
print("Categorical values exist in the columns : ",encoded_need)
encoded_need = encoded_need[:-1]
print(encoded_need)

Unique values of 'Gender' is ['Male' 'Female']
Unique values of 'Customer Type' is ['Loyal Customer' 'disloyal Customer']
Unique values of 'Type of Travel' is ['Personal Travel' 'Business travel']
Unique values of 'Class' is ['Eco Plus' 'Business' 'Eco']
Unique values of 'satisfaction' is ['neutral or dissatisfied' 'satisfied']
Categorical values exist in the columns :  ['Gender', 'Customer Type', 'Type of Travel', 'Class', 'satisfaction']
['Gender', 'Customer Type', 'Type of Travel', 'Class']


In [192]:
max_values = df_copy_cleaned.max()

# หาคอลัมน์ที่มีค่า max = 5
columns_with_max_5 = max_values[max_values == 5].index.tolist()

print("คอลัมน์ที่มีค่า max เท่ากับ 5:", columns_with_max_5)
encoded_all = columns_with_max_5 + encoded_need
print(encoded_all)

คอลัมน์ที่มีค่า max เท่ากับ 5: ['Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking', 'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 'Leg room service', 'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness']
['Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking', 'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 'Leg room service', 'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness', 'Gender', 'Customer Type', 'Type of Travel', 'Class']


In [193]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()

encoded_object = encoder.fit_transform(df_copy_cleaned[encoded_need])

df_encoded_object = pd.DataFrame(encoded_object.toarray(), columns=encoder.get_feature_names_out(encoded_need))
df_encoded_object = df_encoded_object.astype(int)
df_encoded_object

,Gender_Female,Gender_Male,Customer Type_Loyal Customer,Customer Type_disloyal Customer,Type of Travel_Business travel,Type of Travel_Personal Travel,Class_Business,Class_Eco,Class_Eco Plus
0,0,1,1,0,0,1,0,0,1
1,0,1,0,1,1,0,1,0,0
2,1,0,1,0,1,0,1,0,0
3,1,0,1,0,1,0,1,0,0
4,0,1,1,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...
129482,0,1,0,1,1,0,1,0,0
129483,0,1,1,0,1,0,1,0,0
129484,1,0,1,0,0,1,0,1,0
129485,0,1,1,0,1,0,1,0,0


In [194]:
encoder_ob = OneHotEncoder()

encoded = encoder_ob.fit_transform(df_copy_cleaned[encoded_all])

df_encoded = pd.DataFrame(encoded.toarray(), columns=encoder_ob.get_feature_names_out(encoded_all))
df_encoded = df_encoded.astype(int)
df_encoded

,Inflight wifi service_0,Inflight wifi service_1,Inflight wifi service_2,Inflight wifi service_3,Inflight wifi service_4,Inflight wifi service_5,Departure/Arrival time convenient_0,Departure/Arrival time convenient_1,Departure/Arrival time convenient_2,Departure/Arrival time convenient_3,...,Cleanliness_5,Gender_Female,Gender_Male,Customer Type_Loyal Customer,Customer Type_disloyal Customer,Type of Travel_Business travel,Type of Travel_Personal Travel,Class_Business,Class_Eco,Class_Eco Plus
0,0,0,0,1,0,0,0,0,0,0,...,1,0,1,1,0,0,1,0,0,1
1,0,0,0,1,0,0,0,0,1,0,...,0,0,1,0,1,1,0,1,0,0
2,0,0,1,0,0,0,0,0,1,0,...,1,1,0,1,0,1,0,1,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,1,0,1,0,1,0,1,0,0
4,0,0,0,1,0,0,0,0,0,1,...,0,0,1,1,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129482,0,0,0,1,0,0,0,0,0,1,...,0,0,1,0,1,1,0,1,0,0
129483,0,0,0,0,1,0,0,0,0,0,...,0,0,1,1,0,1,0,1,0,0
129484,0,0,1,0,0,0,0,0,0,0,...,0,1,0,1,0,0,1,0,1,0
129485,0,0,0,1,0,0,0,0,0,1,...,0,0,1,1,0,1,0,1,0,0


In [195]:
df_drop_object = df_copy_cleaned.drop(encoded_need, axis=1)
df_drop_object

,Age,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,13,460,3,4,3,1,5,3,5,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,25,235,3,2,3,3,1,3,1,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,26,1142,2,2,2,2,5,5,5,5,4,3,4,4,4,5,0,0.0,satisfied
3,25,562,2,5,5,5,2,2,2,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,61,214,3,3,3,3,4,5,5,3,3,4,4,3,3,3,0,0.0,satisfied
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129482,34,526,3,3,3,1,4,3,4,4,3,2,4,4,5,4,0,0.0,neutral or dissatisfied
129483,23,646,4,4,4,4,4,4,4,4,4,5,5,5,5,4,0,0.0,satisfied
129484,17,828,2,5,1,5,2,1,2,2,4,3,4,5,4,2,0,0.0,neutral or dissatisfied
129485,14,1127,3,3,3,3,4,4,4,4,3,2,5,4,5,4,0,0.0,satisfied


In [196]:
df_drop_object.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129487 entries, 0 to 129486
Data columns (total 19 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Age                                129487 non-null  int64  
 1   Flight Distance                    129487 non-null  int64  
 2   Inflight wifi service              129487 non-null  int64  
 3   Departure/Arrival time convenient  129487 non-null  int64  
 4   Ease of Online booking             129487 non-null  int64  
 5   Gate location                      129487 non-null  int64  
 6   Food and drink                     129487 non-null  int64  
 7   Online boarding                    129487 non-null  int64  
 8   Seat comfort                       129487 non-null  int64  
 9   Inflight entertainment             129487 non-null  int64  
 10  On-board service                   129487 non-null  int64  
 11  Leg room service                   1294

In [197]:
df_drop_dummy = df_copy_cleaned.drop(encoded_all, axis=1)
df_drop_dummy

,Age,Flight Distance,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,13,460,25,18.0,neutral or dissatisfied
1,25,235,1,6.0,neutral or dissatisfied
2,26,1142,0,0.0,satisfied
3,25,562,11,9.0,neutral or dissatisfied
4,61,214,0,0.0,satisfied
...,...,...,...,...,...
129482,34,526,0,0.0,neutral or dissatisfied
129483,23,646,0,0.0,satisfied
129484,17,828,0,0.0,neutral or dissatisfied
129485,14,1127,0,0.0,satisfied


In [198]:
df_drop_dummy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129487 entries, 0 to 129486
Data columns (total 5 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Age                         129487 non-null  int64  
 1   Flight Distance             129487 non-null  int64  
 2   Departure Delay in Minutes  129487 non-null  int64  
 3   Arrival Delay in Minutes    129487 non-null  float64
 4   satisfaction                129487 non-null  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 4.9+ MB


In [199]:
# ตรวจสอบขนาดของ df_drop_dummy และ df_encoded ก่อน
print("ขนาดของ df_drop_object:", df_drop_object.shape)
print("ขนาดของ df_encoded:", df_encoded_object.shape)

df_final_object = pd.concat([df_encoded_object, df_drop_object], axis=1)
print("ขนาดของ df_final:", df_final_object.shape)

ขนาดของ df_drop_object: (129487, 19)
ขนาดของ df_encoded: (129487, 9)
ขนาดของ df_final: (129487, 28)


In [200]:
# ตรวจสอบขนาดของ df_drop_dummy และ df_encoded ก่อน
print("ขนาดของ df_drop_dummy:", df_drop_dummy.shape)
print("ขนาดของ df_encoded:", df_encoded.shape)

df_final = pd.concat([df_encoded, df_drop_dummy], axis=1)
print("ขนาดของ df_final:", df_final.shape)


ขนาดของ df_drop_dummy: (129487, 5)
ขนาดของ df_encoded: (129487, 92)
ขนาดของ df_final: (129487, 97)


In [201]:
df_final_object

,Gender_Female,Gender_Male,Customer Type_Loyal Customer,Customer Type_disloyal Customer,Type of Travel_Business travel,Type of Travel_Personal Travel,Class_Business,Class_Eco,Class_Eco Plus,Age,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,1,1,0,0,1,0,0,1,13,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,0,1,0,1,1,0,1,0,0,25,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,1,0,1,0,1,0,1,0,0,26,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,1,0,1,0,1,0,1,0,0,25,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,0,1,1,0,1,0,1,0,0,61,...,3,3,4,4,3,3,3,0,0.0,satisfied
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129482,0,1,0,1,1,0,1,0,0,34,...,4,3,2,4,4,5,4,0,0.0,neutral or dissatisfied
129483,0,1,1,0,1,0,1,0,0,23,...,4,4,5,5,5,5,4,0,0.0,satisfied
129484,1,0,1,0,0,1,0,1,0,17,...,2,4,3,4,5,4,2,0,0.0,neutral or dissatisfied
129485,0,1,1,0,1,0,1,0,0,14,...,4,3,2,5,4,5,4,0,0.0,satisfied


In [202]:
df_final

,Inflight wifi service_0,Inflight wifi service_1,Inflight wifi service_2,Inflight wifi service_3,Inflight wifi service_4,Inflight wifi service_5,Departure/Arrival time convenient_0,Departure/Arrival time convenient_1,Departure/Arrival time convenient_2,Departure/Arrival time convenient_3,...,Type of Travel_Business travel,Type of Travel_Personal Travel,Class_Business,Class_Eco,Class_Eco Plus,Age,Flight Distance,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,1,13,460,25,18.0,neutral or dissatisfied
1,0,0,0,1,0,0,0,0,1,0,...,1,0,1,0,0,25,235,1,6.0,neutral or dissatisfied
2,0,0,1,0,0,0,0,0,1,0,...,1,0,1,0,0,26,1142,0,0.0,satisfied
3,0,0,1,0,0,0,0,0,0,0,...,1,0,1,0,0,25,562,11,9.0,neutral or dissatisfied
4,0,0,0,1,0,0,0,0,0,1,...,1,0,1,0,0,61,214,0,0.0,satisfied
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129482,0,0,0,1,0,0,0,0,0,1,...,1,0,1,0,0,34,526,0,0.0,neutral or dissatisfied
129483,0,0,0,0,1,0,0,0,0,0,...,1,0,1,0,0,23,646,0,0.0,satisfied
129484,0,0,1,0,0,0,0,0,0,0,...,0,1,0,1,0,17,828,0,0.0,neutral or dissatisfied
129485,0,0,0,1,0,0,0,0,0,1,...,1,0,1,0,0,14,1127,0,0.0,satisfied


## Save Data

In [203]:
df_final.to_csv('../datasets/data_dummy.csv', index=False)
df_final_object.to_csv('../datasets/data_dummy_object.csv', index=False)